# Ray Tasks Revisited

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

The [Ray Crash Course](../ray-crash-course/00-Ray-Crash-Course-Overview.ipynb) introduced the core concepts of Ray's API and how they parallelize work. Specifically, we learned how to define Ray _tasks_ and _actors_, run them, and retrieve the results. 

This lesson explores Ray tasks in greater depth, including the following:

* How task dependencies are handled automatically by Ray
* Usage patterns for `ray.get()` and `ray.wait()`
* Specifying limits on the number of invocations and retries on failure
* An exploration of task granularity considerations
* Profiling tasks

In [9]:
!../tools/start-ray.sh


Ray already running or successfully started


In [10]:
import ray, time, os, sys 
import numpy as np 
sys.path.append("..")
from util.printing import pd, pnd  # convenience methods for printing results.

In [11]:
ray.init(address='auto', ignore_reinit_error=True)

2020-05-24 08:29:46,641	ERROR worker.py:700 -- Calling ray.init() again after it has already been called.


The Ray Dashboard URL.

The following URL will only work when running this lesson on your laptop. When using the Anyscale platform, use the URL provided by your instructor to access the Ray Dashboard.

In [12]:
print(f'Ray Dashboard: http://{ray.get_webui_url()}')

Ray Dashboard: http://localhost:8265


## Ray Task Dependencies

Let's define a few remote tasks, which will have _dependency_ relationships. We'll learn how Ray handles these dependent, asynchronous computations.

One task will return a random NumPy array of some size `n` and the other task will add two such arrays. We'll also add a sleep time, one tenth the size of `n` to simulate expensive computation.

> **Note:** Dependencies and how Ray implements handling of them are explored in depth in the [03: Ray Internals](03-Ray-Internals.ipynb) lesson. 

In [13]:
@ray.remote
def make_array(n):
    time.sleep(n/10.0)
    return np.random.standard_normal(n)

Now define a task that can add two NumPy arrays together. The arrays need to be the same size, but we'll ignore any checking for this requirement.

In [14]:
@ray.remote
def add_arrays(a1, a2):
    time.sleep(a1.size/10.0)
    return np.add(a1, a2)

Now lets use them!

In [15]:
start = time.time()
id1 = make_array.remote(20)
id2 = make_array.remote(20)
id3 = add_arrays.remote(id1, id2)
print(ray.get(id3))
pd(time.time() - start, prefix="Total time:")

[ 0.50483788  0.46171886  0.85278257  1.09167304 -0.48461469 -1.11997619
 -0.42866996 -1.41434347 -2.36917476 -0.41429467  1.17602985 -0.72150781
  0.87405619 -2.94742449  2.42910966 -3.41100176  2.07784609  0.29566728
 -2.42730875  0.9878451 ]
Total time: duration:  4.048 seconds


Something subtle and "magical" happened here; when we called `add_arrays`, we didn't need to call `ray.get()` first for `id1` and `id2`, since `add_arrays` expects NumPy arrays. Because `add_arrays` is a Ray task, Ray automatically does the extraction for us, so we can write code that looks more natural.

Furthermore, note that the `add_arrays` task effectively depends on the outputs of the two `make_array` tasks. Ray won't run `add_arrays` until the other tasks are finished. Hence, Ray automatically handles task dependencies for us.

This is why the elapsed time is about 4 seconds. We used a size of 20, so we slept 2 seconds in each call to `make_array`, but those happened in parallel, _followed_ by a second sleep of 2 seconds in `add_arrays`.

Even though three task invocations occurred, we only used one call to `ray.get()`, when we actually needed the final results. Eliminating unnecessary `ray.get()` calls helps avoid forcing tasks to become synchronous when they could be asynchronous. So, keep these two key points in mind:

* _Don't ask for results you don't need._
* _Don't ask for the results you need until you really need them._

We don't need to see the objects for `id1` and `id2`. We only need the final array for `id3`.

## Using ray.wait() with ray.get()

Here is an idiomatic way to use `ray.get()`, where we fire all five asynchronous tasks at once, then ask for all the results at once with `ray.get()`:

In [16]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]
for array in ray.get(added_array_ids):
    print(f'{array.size}: {array}')
pd(time.time() - start, prefix="Total time:")

0: []
10: [-1.14780127  1.46203251  0.95254106 -1.60007792  1.24589508  1.52212463
  0.83314686 -0.63177348 -3.41808833  0.41103595]
20: [ 1.97026322 -0.58286115 -0.01731141  0.26751702 -1.87688826 -2.04411651
 -4.11452954  0.77451951 -0.34220767 -2.99170328  2.71962818 -0.8709754
 -0.05093316  0.97647008  0.78567023  0.77135754  0.39747319 -0.6082992
  0.09805494  1.34347914]
30: [ 4.48256558 -0.77314292 -0.41535901 -1.04026051  2.76468215  0.84463148
 -0.46673385 -3.1659171  -0.43187332 -2.89400918 -3.75219219 -2.04132911
  0.22747529 -1.75011107  1.12279133 -2.79438777  2.12019141 -1.70345364
 -2.35820991  0.45702111  0.78809585 -0.25348494  2.14764278  3.55910413
  4.21153071  1.03542209  1.04301996  2.87307274  0.77067346 -2.782334  ]
40: [ 1.56611342 -3.0757839   1.38706115 -1.28474306 -1.92853324 -1.55633054
  1.19755407  1.77136333  0.11860839 -1.2579904   0.49718663  0.11480678
 -1.71486781  0.07046719 -5.81731278  1.51562054  1.55728854  2.35924347
  4.06099667 -0.41422313 -0

This takes about eight seconds, four seconds for the longest invocation invocation of `make_array`, `make_array(4)`  , and four seconds with longest invocation of `add_arrays`, when passed the results of `make_array(4)`. 

We did the right thing inside each list comprehension. We started the asynchronous tasks all at once and allowed Ray to handle the dependencies. Then we waited on one `ray.get()` call for all the output. 

However, what you see is no output and then everything is suddenly printed at once after eight seconds.

There are two fundamental problems with the way we've used `ray.get()` so far:

1. There's no timeout, in case something gets "hung".
2. We have to wait for _all_ the objects to be available before `ray.get()` returns.

The ability to specify a timeout is essential in production code as a defensive measure. Many potential problems could happen in a real production system, any one of which could cause the task we're waiting on to take an abnormally long time to complete or never complete. Our application would be deadlocked waiting on this task. Hence, it's **strongly recommended** in production software to always use timeouts on blocking calls, so that the application can attempt some sort of recovery in situations like this, or at least report the error and "degrade gracefully".

Actually, there _is_ a `timeout=<value>` option you can pass to `ray.get()` ([documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.get)), but it will most likely be removed in a future release of Ray. Why remove it if timeouts are important? This change will simplify the implementation of `ray.get()` and encourage the use of `ray.wait()` for waiting ([documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.wait)) instead, followed by using `ray.get()` to retrieve values for tasks that `ray.wait()` tells us are finished. 

Using `ray.wait()` is also the way to fix the second problem with using `ray.get()` by itself, that we have to wait for all tasks to finish before we get any values back. Some of those tasks finish more quickly in our contrived example. We would like to process those results as soon as they are available, even while others continue to run. We'll use `ray.wait()` for this purpose.

Therefore, while `ray.get()` is simple and convenient, for _production code_, we recommend using `ray.wait()`, **with** timeouts, for blocking on running tasks. Then use `ray.get()` to retrieve values of completed tasks. 

Here is the previous example rewritten to use `ray.wait()`:

In [17]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]

arrays = []
waiting_ids = list(added_array_ids)        # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as one of them completes,
    #   3. tell it wait up to 10 seconds before timing out.
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=1, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_ids), time.time() - start))
    new_arrays = ray.get(ready_ids)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   1 completed tasks. (elapsed time:  0.005)
0: []
Returned   1 completed tasks. (elapsed time:  2.010)
10: [-0.96591702  1.97437847  1.21454945  0.87932681 -3.34879174  0.33058646
  0.878718   -1.82498122 -0.08776717 -1.63413767]
Returned   1 completed tasks. (elapsed time:  4.009)
20: [-2.47544295  1.5556616  -0.08105603 -0.12523109 -0.64303378  0.36511514
  0.552237    1.5821803  -1.60792174  1.10878674  2.1467292   0.19064597
  0.6012799  -4.19823563 -0.47513118 -1.76114334  2.38011003  0.22345365
 -0.11730344  0.03708495]
Returned   1 completed tasks. (elapsed time:  6.010)
30: [-0.43739135 -1.66788541 -0.15671976  4.3718916  -2.94715611 -0.81067895
 -3.7216232  -0.99616618  0.06611263 -0.02138862 -0.23026996  2.69896496
 -0.4703861   2.96805252 -2.30356736 -2.36154522 -0.09865887  0.17199027
  3.52559086 -0.60587127 -2.28313268 -2.52511323 -2.8576861  -2.38871185
 -0.08372164  0.68943195 -2.24008962  0.41358353  3.55949352 -0.91064845]
Returned   1 completed tasks. (elaps

Now it still takes about 8 seconds to complete, 4 seconds for the longest invocation of `make_array` and 4 seconds for the invocation of `add_arrays`, but since the others complete more quickly, we see their results as soon as they become available, at 0, 2, 4, and 6 second intervals.

> **Warning:** For each call to `ray.wait()` in a loop like this, it's important to remove the ids that have completed. Otherwise, `ray.wait()` will return immediately with the same list containg the first completed item, over and over again; you'll loop forever!! Resetting the list is easy, since the second list returned by `ray.wait()` is the rest of the items that are still running. So, that's what we use.

Now let's try it with `num_returns = 2`:

In [18]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]

arrays = []
waiting_ids = list(added_array_ids)        # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as TWO of them complete,
    #   3. tell it wait up to 10 seconds before timing out.
    return_n = 2 if len(waiting_ids) > 1 else 1
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=return_n, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_ids), time.time() - start))
    new_arrays = ray.get(ready_ids)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   2 completed tasks. (elapsed time:  2.011)
0: []
10: [ 0.28349942 -0.83427243 -4.17891534  0.12914285  0.54454409 -0.166779
  5.9331363  -2.28770148  0.14925713  2.16425932]
Returned   2 completed tasks. (elapsed time:  6.009)
20: [-3.16686106 -3.58254814 -0.4072525   2.46390315  3.1726024  -2.03857785
  2.16555245 -0.69060848  2.43357763 -0.24435906 -2.21326882  3.4150321
  4.55881046  2.7551483  -0.4218604   0.05562842 -0.11238671  1.76013854
  3.09458086  1.13519071]
30: [ 2.77374833  0.69376923 -3.4201408   0.77189893  1.86332426 -1.78671821
 -2.64248851 -0.89984086  1.07674811  2.13744037 -0.43371648  1.13033464
  0.20365627  0.45492943  0.50629758  2.45098619 -2.56824965 -0.79346502
 -0.02261038  4.11459999 -0.07724809 -2.4474074   1.44807345 -1.8052588
 -1.04384321 -0.55811605 -0.66708765  1.63967649  1.9115157   0.07157019]
Returned   1 completed tasks. (elapsed time:  8.013)
40: [-1.86516842e+00  2.23122355e+00  1.27202574e+00 -3.74044802e+00
 -1.89776818e+00  1.7952

Now we get two at a time output. Note that we don't actually pass `num_returns=2` every time. If you ask for more items than the length of the input list, you get an error. So, we compute `num_returns`, using `2` except when there's only one task to wait on, in which case we use `1`. So, in fact, the output for `40` was a single task result, because we started with `5` and processed two at a time.

 For a longer discussion on `ray.wait()`, see [this blog post](https://medium.com/distributed-computing-with-ray/ray-tips-and-tricks-part-i-ray-wait-9ed7a0b9836d).

## Exercise 1

The following cell is identical to the last one. Modify it to use a timeout of `2.5` seconds, shorter than our longest tasks. What happens now? Try using other times.

See the [solutions notebook](solutions/Advanced-Ray-Solutions.ipynb) for a discussion of this exercise and the subsequent exercises.

In [19]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]

arrays = []
waiting_ids = list(added_array_ids)        # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as TWO of them complete,
    #   3. tell it wait up to 10 seconds before timing out.
    return_n = 2 if len(waiting_ids) > 1 else 1
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=return_n, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_ids), time.time() - start))
    new_arrays = ray.get(ready_ids)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   2 completed tasks. (elapsed time:  2.007)
0: []
10: [ 2.44256673 -0.54742863  2.25232475 -0.97330648  1.15525133  2.56258169
 -1.15692063  0.90050818 -2.08677396  3.26906464]
Returned   2 completed tasks. (elapsed time:  6.014)
20: [-1.86133288 -0.41284786  0.70418553 -0.73606234 -3.37550143  2.20113229
 -2.35618131  3.79920562  3.82532414  2.6156344   0.34072562 -3.20958041
  0.18719492  0.48991589  2.79347686 -2.50215841  2.13398481  2.09283493
 -2.72932319 -4.53102084]
30: [-0.84925251  0.62975546  0.21057678  1.34546169  0.14510744  4.86356391
 -0.46019643 -1.4354555   1.20573535  0.19379843  0.54979878 -0.2126731
  2.88883609  0.07114011 -0.95941639  0.54881987  3.69712145  1.39060227
 -8.07363567 -1.39693323 -0.24644035 -1.41617052 -1.46519483  0.85203503
  2.11861912  0.85741316 -0.7195816  -0.32275792  0.84406495  1.35666557]
Returned   1 completed tasks. (elapsed time:  8.013)
40: [ 1.96889043 -3.40609107 -1.40344103  1.18276346 -0.25153568 -0.32581959
  0.90495186 

In conclusion:

> **Tips:**
>
> 1. Use `ray.wait()` with a timeout to wait for one or more running tasks. Then use `ray.get()` to retrieve the values for the finished tasks.
> 2. When looping over calls to `ray.wait()` with a list of object ids for running tasks, remove the previously-completed and retrieved objects from the list.
> 3. Don't ask for results you don't need.
> 4. Don't ask for the results you need until you really need them.

## Exercise 3

Let's practice converting a slow loop to Ray, including using `ray.wait()`. Change the function to be a Ray task. Change the invocations to use the `ray.wait()` idiom. You can just use the default values for `num_returns` and `timeout` if you want. The second cell uses `assert` statements to check your work.

In [22]:
def slow_square(n):
    time.sleep(n)
    return n*n

start = time.time()
squares = [slow_square(n) for n in range(4)]
for square in squares:
    print (f'finished: {square}')
duration = time.time() - start

finished: 0
finished: 1
finished: 4
finished: 9


In [23]:
assert squares == [0, 1, 4, 9], f'Did you use ray.get() to retrieve the values? squares = {squares}'
assert duration < 4.1, f'Did you use Ray to parallelize the work? duration = {duration}' 

AssertionError: Did you use Ray to parallelize the work? duration = 6.007376194000244

## Limiting Task Invocations and Retries on Failure

> **Note:** This feature may change in a future version of Ray. See the latest details in the [Ray documentation](https://docs.ray.io/en/latest/package-ref.html#ray.remote). 

Two options you can pass to `ray.remote` when defining a task affect how often it can be invoked and retrying on failure:

* `max_calls`: This specifies the maximum number of times that a given worker can execute the given remote function before it must exit. This can be used to address memory leaks in third-party libraries or to reclaim resources that cannot easily be released, e.g., GPU memory that was acquired by TensorFlow. By default this is infinite.
* `max_retries`: This specifies the maximum number of times that the remote function should be rerun when the worker process executing it crashes unexpectedly. The minimum valid value is 0, the default is 4, and a value of -1 indicates infinite retries are allowed.

Example:

```python
@ray.remote(max_calls=10000, max_retries=10)
def foo():
    pass
```

See the [ray.remote()](https://docs.ray.io/en/latest/package-ref.html#ray.remote) documentation for all the keyword arguments supported.

### Overriding with config()

Remote task and actor objects returned by `@ray.remote` can also be dynamically modified with the same arguments supported by `ray.remote()` using `options()` as in the following examples:

```python
@ray.remote(num_gpus=1, max_calls=1, num_return_vals=2)
def f():
    return 1, 2
g = f.options(num_gpus=2, max_calls=None)
```

## What Is the Optimal Task Granularity

How fine-grained should Ray tasks be? There's no fixed rule of thumb, but Ray clearly adds some overhead for task management and using object stores in a cluster. Therefore, it makes sense that tasks which are too small will perform poorly.

We'll explore this topic over several more lessons, but for now, let's get a sense of the overhead while running in your setup.

We'll continue to use NumPy arrays to create "load", but remove the `sleep` calls:

In [24]:
def noop(n):
    return n

def local_make_array(n):
    return np.random.standard_normal(n)

@ray.remote
def remote_make_array(n):
    return local_make_array(n)

Let's do `trials` runs for each experiment, to average out background noise:

In [25]:
trials=100

First, let's use `noop` to baseline local function calls. Note that we call `print` for the duration, rathern than `pd`, because the overhead is so low the `pd` formatting will print `0.000`:

In [26]:
start = time.time()
[noop(t) for t in range(trials)]
print(f'{time.time() - start} seconds')

0.00011110305786132812 seconds


Let's try the same run with `local_make_array(n)` for `n = 100000`:

In [27]:
start = time.time()
[local_make_array(100000) for _ in range(trials)]
print(f'{time.time() - start} seconds')

0.26585912704467773 seconds


So, we can safely ignore the "noop" overhead for now. For completeness, here's what happens with remote execution:

In [28]:
start = time.time()
ids = [remote_make_array.remote(100000) for _ in range(trials)]
ray.get(ids)
print(f'{time.time() - start} seconds')

0.15054988861083984 seconds


For arrays of 100000, using Ray is faster (at least on this test machine). The benefits of parallel computation, rather than synchronous, already outweight the Ray overhead.

So, let's run some trials with increasingly large array sizes, to compare the performance with local vs. remote execution. First, we'll set up `matplotlib`:

In [29]:
local_durations = []
remote_durations = []
# These n values were determined by experimentation on this test machine. 
# If you are using an old machine, and this cell takes a long time to execute,
# you could set the `trials` value above to a smaller number. 
ns = [i*(10**j) for j in range(2,5) for i in [1,2,3,5,8]]
for n in ns:
    start_local = time.time()
    [local_make_array(n) for _ in range(trials)]
    local_durations.append(time.time() - start_local)
    
    start_remote = time.time()
    ids = [remote_make_array.remote(n) for _ in range(trials)]
    ray.get(ids)
    remote_durations.append(time.time() - start_remote)
(ns, local_durations, remote_durations)

([100,
  200,
  300,
  500,
  800,
  1000,
  2000,
  3000,
  5000,
  8000,
  10000,
  20000,
  30000,
  50000,
  80000],
 [0.001184225082397461,
  0.0010838508605957031,
  0.0010738372802734375,
  0.0012750625610351562,
  0.002151012420654297,
  0.0025620460510253906,
  0.006283998489379883,
  0.007977008819580078,
  0.01529693603515625,
  0.02200913429260254,
  0.029201269149780273,
  0.0530393123626709,
  0.08022308349609375,
  0.15346312522888184,
  0.21689319610595703],
 [0.025315046310424805,
  0.022243261337280273,
  0.017681121826171875,
  0.017520904541015625,
  0.01759815216064453,
  0.02542901039123535,
  0.025136709213256836,
  0.040142059326171875,
  0.0230557918548584,
  0.025069236755371094,
  0.025058984756469727,
  0.03670096397399902,
  0.04892897605895996,
  0.0855402946472168,
  0.10716700553894043])

In [30]:
import numpy as np

from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show

import bokeh.io
# The next two lines prevent Bokeh from opening the graph in a new window.
bokeh.io.reset_output()
bokeh.io.output_notebook()

tooltips = [
    ("name", "$name"),
    ("array size", "$x"),
    ("time", "$y")]
p1 = figure(x_axis_type="log", y_axis_type="log", title="Execution Times", tooltips=tooltips)
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = 'array size'
p1.yaxis.axis_label = 'time'

p1.line(ns, local_durations, color='#A6CEE3', legend_label='local', name='local')
p1.circle(ns, local_durations, color='darkgrey', size=4)
p1.line(ns, remote_durations, color='#B2DF8A', legend_label='remote', name='remote')
p1.square(ns, remote_durations, color='darkgrey', size=4)
p1.legend.location = "top_left"

show(gridplot([[p1]], plot_width=800, plot_height=400))

Loading BokehJS ...

Can't see the graph? Here's [one from a prior test run](../images/Execution-Times-Local-v-Remote.png). Your results may look a lot different!

Let's confirm what the graph shows as the crossing point:

In [31]:
i=0
while i < len(ns) and local_durations[i] < remote_durations[i]:
    i=i+1
print('The Ray times are faster starting at n = {:d}, local = {:6.3f} vs. remote = {:6.3f}'.format(
    ns[i], local_durations[i], remote_durations[i]))

The Ray times are faster starting at n = 10000, local =  0.029 vs. remote =  0.025


## Profiling Tasks with ray.timeline()

Sometimes you need to debug performance problems in Ray tasks. Calling `ray.timeline(file)` ([documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.timeline)) captures profiling information for subsequent task execution to the specified file. Afterwards, you can view the data in a Chrome web browser. The format used is unique to Chrome, so Chrome has be used to view the data.

Let's try it with our `make_array` and `add_arrays` methods in the following code. First some potential cleanup:

In [ ]:
timeline_file = 'task-timeline.txt' # Will be found in the same directory as this notebook.
if os.path.isfile(timeline_file):   # Delete old one, if an old one exists already.
    os.remove(timeline_file)

In [32]:
ray.timeline(timeline_file)
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]
for array in ray.get(added_array_ids):
    print(f'{array.size}: {array}')
pd(time.time() - start, prefix="Total time:")

0: []
10: [ 0.78238557 -2.41901674 -2.03579309  2.58167239  0.3303791   1.58021312
 -0.45391349  0.41356556  1.36938425 -0.13650164]
20: [ 0.13908276 -0.44758179 -0.60030316 -1.34394392 -3.25524327  0.73864255
 -3.60323706 -0.17926937  2.41839768 -0.50048685  0.06691712  0.62651438
  0.098529    1.68545524 -0.88790412  0.77811573 -2.65393602 -0.30846447
 -2.28734575 -0.08750034]
30: [-0.62454115 -0.43276699  0.57409673  0.04304678 -0.65722823  0.47686011
  1.24530921 -0.11169047 -3.85563739  3.08454734 -3.95250244 -1.69908259
  2.23093746 -1.97685132  1.84540515  1.71069364  2.52587682  2.85198268
  2.02482653 -1.24358834  1.92466168 -1.10929292 -1.06639084 -0.81799815
 -2.41727479 -0.03795381  0.95209046  0.70887273 -0.49196319 -3.94863565]
40: [-0.93648987  1.02349725  0.13708982 -0.617854   -1.49988038  3.12820247
 -0.81083243  5.02016961 -0.51705818  3.59010015 -2.09396965  1.26067538
  0.63387643  0.92637715  1.25414245 -1.10428506 -0.77876509 -4.92345056
 -1.83180414  2.98921551 

Now, to view the data:

1. Open Chrome and enter chrome://tracing.
2. Click the _Load_ button to load the `task-timeline.txt` file, which will be in this notebook's directory. 
3. To zoom in or out, click the "asymmetric" up-down arrow button. Then hold the mouse button in the graph and roll the mouse scroll wheel up or down. (On a laptop trackpad, press and hold, then use another finger to slide up and down.)
4. To move around, click the crossed arrow and drag a section in view. 
5. Click on a box in the timeline to see details about it. 

Look for blocks corresponding to long-running tasks and look for idle periods, which reflect processing outside the context of Ray.

Here is a screen grab profiling the previous code, zoomed in on one block of tasks and with one task selected. Note the processes shown on the left for drivers (more than one notebook was running at this time) and workers.

![Ray Trace Example](../images/Ray-Trace-Example.png)

In [33]:
ray.shutdown()  # "Undo ray.init()". Terminate all the processes started in this notebook.